In [17]:
from glob import glob
from os import path
import pandas as pd


List all files to explore

In [18]:
glob(path.join('DB/institutes','*.csv'))

['DB/institutes\\institutes_academiainstitute.csv',
 'DB/institutes\\institutes_basicservicesupplierinstitute.csv',
 'DB/institutes\\institutes_financialinstitute.csv',
 'DB/institutes\\institutes_institute.csv',
 'DB/institutes\\institutes_investorinstitute.csv',
 'DB/institutes\\institutes_technicalassistanceinstitute.csv']

Ingest all files in the selected directory

In [19]:
file_names = glob(path.join('DB/institutes','*.csv'))
dfs = [pd.read_csv(fn) for fn in file_names]
institutes_df = pd.concat(dfs)

print(institutes_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123 entries, 0 to 12
Data columns (total 37 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                123 non-null    int64  
 1   legal_status                      47 non-null     object 
 2   research_areas                    12 non-null     object 
 3   focus_areas                       32 non-null     object 
 4   impact_metrics                    25 non-null     object 
 5   regulated                         9 non-null      object 
 6   mixmarket_profile                 0 non-null      float64
 7   services_offered                  11 non-null     object 
 8   non_financial_programs            11 non-null     object 
 9   main_funding_sources              11 non-null     object 
 10  current_financial_needs_currency  11 non-null     object 
 11  current_financial_needs           0 non-null      float64
 12  funding_n

In [20]:
print(list(institutes_df.columns.values))

['id', 'legal_status', 'research_areas', 'focus_areas', 'impact_metrics', 'regulated', 'mixmarket_profile', 'services_offered', 'non_financial_programs', 'main_funding_sources', 'current_financial_needs_currency', 'current_financial_needs', 'funding_needed', 'savings', 'name', 'short_name', 'website_url', 'date_of_establishment', 'total_number_of_staff', 'object_id', 'address_id', 'content_type_id', 'type_id', 'mission', 'vision', 'approved', 'avatar', 'portfolio_size', 'funding_institution_type', 'funding_max', 'funding_max_currency', 'funding_min', 'funding_min_currency', 'funding_offered', 'term_max', 'term_min', 'services']


We will export the DataFrame to have a look using Excel. 

In [21]:
#institutes_df.to_csv('institutes_df.csv')

According to this dataset, some fielda are not relevant to find the nearest neighbors (matches).
This fields are:
- id: this field is an identification number.
- website_url: this field is a webpage.
- date_of_establishment: this field is a date of establishment **might not be relevant. Or is it?**
- address_id: this field looks like a running number. **Is it relevant?**
- mission: this field is an open text. We could apply NLP to identify topics. **Maybe in a version 2 of the tool.**
- vision: this field is an open text. We could apply NLP to identify topics. **Maybe in a version 2 of the tool.**
- avatar: this field is an image.

In [23]:
institutes_df_short = institutes_df.drop(['website_url','date_of_establishment','address_id','mission','vision','avatar'], axis=1)

Replace all blancs (white spaces) with NaN

In [26]:
institutes_df_short = institutes_df_short.replace(r'^\s*$', np.nan, regex=True)
#print(institutes_df_short.head())

Replace all empty strings {} with NaN

In [27]:
institutes_df_short = institutes_df_short.replace('{}', np.nan, regex=True)

In [52]:
# summarize the number of rows with missing values for each column
for i in range(institutes_df_short.shape[1]):
    # count number of rows with missing values
    n_miss = institutes_df_short.loc[i].isnull().sum()
    perc = n_miss / institutes_df_short.shape[0] * 100
    #print('> %d, Missing: %d (%.1f%%)' % (i, n_miss, perc))

Let's check in Excel...

In [28]:
institutes_df_short.to_csv('institutes_df_short.csv')

In [53]:
institutes_df_short.head()

,id,legal_status,research_areas,focus_areas,impact_metrics,regulated,mixmarket_profile,services_offered,non_financial_programs,main_funding_sources,...,portfolio_size,funding_institution_type,funding_max,funding_max_currency,funding_min,funding_min_currency,funding_offered,term_max,term_min,services
0,1,University,"{""Food security"",Water,Housing,Clothing,Sanita...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,R&D of private company,"{""Food security"",Water,Housing,Clothing}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,R&D of NGO,"{""Food security"",Water,Housing,Clothing,Sanita...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,University,"{""Food security"",Water}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Research institute,"{""Food security"",Water,Housing,Clothing,Sanita...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


To devide the categorical data into dummy variables we need to convert only the variables of type 'object'

In [31]:
# Make a list of column names for variables of the type object
object_variables = list(institutes_df_short.select_dtypes(include=['object']).columns)
print(object_variables)

['legal_status', 'research_areas', 'focus_areas', 'impact_metrics', 'regulated', 'services_offered', 'non_financial_programs', 'main_funding_sources', 'current_financial_needs_currency', 'funding_needed', 'savings', 'name', 'short_name', 'type_id', 'approved', 'portfolio_size', 'funding_institution_type', 'funding_max_currency', 'funding_min_currency', 'funding_offered', 'services']


In [41]:
# Let's check the column names for variables of the type int64

print(list(institutes_df_short.select_dtypes(include=['int64']).columns))

['id']


In [42]:
# Let's check the column names for variables of the type float

print(list(institutes_df_short.select_dtypes(include=['float']).columns))

['mixmarket_profile', 'current_financial_needs', 'total_number_of_staff', 'object_id', 'content_type_id', 'funding_max', 'funding_min', 'term_max', 'term_min']


In [36]:
X = pd.get_dummies(institutes_df_short, columns=object_variables)

Let's check in Excel...

In [37]:
X.to_csv('X.csv')

In [43]:
# print(list(X.columns.values))